In [15]:
import re
import json
from pprint import pprint
import pandas as pd

In [16]:
with open('files/master.txt') as f:
    lines = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: 'files/master.txt'

In [17]:
def find_name(x):
    if re.match('(.*)[\(\)]? \d{1,2}:\d{1,2} [AP]M', x) != None and 'Today at' not in x and 'reply' not in x and 'replies' not in x:
        return x

def find_post(x):
    x = x.encode('unicode_escape').decode()
    if re.match('.*(Wordle \d{1,3} \d\/\d[\*]?)', x) != None:
        return re.match('.*(Wordle \d{1,3} \d\/\d[\*]?)', x)[1]

def find_squares(x):
    if '::' in x and 'square' in x:
        x = x.encode('unicode_escape').decode()

        s = re.match('.*(Wordle \d{1,3} \d\/\d[\*]?)(.*?)((:large_green_square:){5})', x).groups()

        s = ''.join(list(filter(lambda a: 'square' in a and a != ':large_green_square:', s))).strip()
        return s

msg =  'Wordle 206 3/6*\n:white_large_square::white_large_square::white_large_square::large_yellow_square::large_yellow_square:\n:white_large_square::large_green_square::large_green_square::large_green_square::white_large_square:\n:large_green_square::large_green_square::large_green_square::large_green_square::large_green_square:\n\nChris Shibe  12:51 PM\n'

m = 'Scott Lohbauer (he/him/his)  10:47 AM\noh boy\n10:48\nMe yesterday: I hate it\nWordle 221 4/6*\n:white_large_square::large_yellow_square::white_large_square::white_large_square::white_large_square:\n:large_yellow_square::white_large_square::white_large_square::white_large_square::white_large_square:\n:white_large_square::white_large_square::large_green_square::white_large_square::large_green_square:\n:large_green_square::large_green_square::large_green_square::large_green_square::large_green_square:\n:grimacing:\n1\n\n\nScott Lohbauer (he/him/his)  8:45 AM\n'
find_post(m)


'Wordle 221 4/6*'

In [19]:

posts_filtered = []
name = 'None'
msg = []
new_name = None
wordle = 'None None None'
new_wordle = None
squares = []
raw = []
for i,l in enumerate(['None None 00:00 AM'] + lines):
    msg.append(l)
    
    if find_name(l) != None: new_name = find_name(l)

    if (new_name != None and new_name != name):     
      msg = ''.join(msg)
      raw.append(name+msg)
      if find_post(msg) != None :
        squares = find_squares(msg)
        wordle = find_post(msg)
        tries = wordle.split(' ')[2].split('/')[0]
        posts_filtered.append({'name': re.sub('\(.*\) | \d{1,2}:\d{1,2} [AP]M.*', '', name).strip(),
                               'first_name': name.split(' ')[0],
                               'post': ''.join(name+msg),
                               'wordle': wordle.split(' ')[1],
                               'tries': tries,
                               'board': re.sub('\\\\n','\\n',squares),
                               'hardmode': True if '*' in wordle else False})

      name = new_name
      msg = []
    
# write out last game
if (new_name != None):     
      msg = ''.join(msg)
      raw.append(new_name+msg)
      if find_post(msg) != None :
        squares = find_squares(msg)
        wordle = find_post(msg)
        tries = wordle.split(' ')[2].split('/')[0]
        posts_filtered.append({'name': re.sub('\(.*\) | \d{1,2}:\d{1,2} [AP]M.*', '', name).strip(),
                               'first_name': new_name.split(' ')[0],
                               'post': ''.join(new_name+msg),
                               'wordle': wordle.split(' ')[1],
                               'tries': tries,
                               'board': re.sub('\\\\n','\\n',squares),
                               'hardmode': True if '*' in wordle else False})
  



In [13]:
with open('wordles.json', 'w') as f:
    json.dump(posts_filtered, f)

In [10]:
posts_df = pd.DataFrame(posts_filtered)
posts_df['name'] = posts_df['first_name']
posts_df.drop(columns = ['post'],inplace = True)
posts_df

,name,first_name,wordle,tries,board,hardmode
0,Scott,Scott,205,5,\n:white_large_square::white_large_square::lar...,True
1,Scott,Scott,206,3,\n:white_large_square::white_large_square::whi...,True
2,Chris,Chris,216,5,\n:white_large_square::white_large_square::whi...,False
3,Scott,Scott,216,3,\n:white_large_square::white_large_square::lar...,True
4,Koba,Koba,221,2,\n:large_green_square::black_large_square::lar...,False
...,...,...,...,...,...,...
166,Don,Don,254,3,\n:large_green_square::black_large_square::lar...,True
167,Chris,Chris,255,5,\n:large_yellow_square::black_large_square::bl...,False
168,Don,Don,255,5,\n:black_large_square::large_yellow_square::la...,True
169,Koba,Koba,255,4,\n:black_large_square::large_yellow_square::bl...,True


In [12]:
with open('wordles.json', 'w') as f:
    json.dump(posts_df.to_json(orient='records'), f)


In [14]:
posts_df.to_json('wordles.json',orient='records')